In [1]:
import json
import os
import geopandas as gpd
from shapely.geometry import Point, LineString,box,MultiLineString
import datetime
import time
import numpy as np
from shapely import wkt
import pandas as pd

In [2]:
raw_data_path = "raw_data/"
#sector_name  = "sector_67Y"
sector_name = "sector_w_esmm"
processed_data_path = "processed_data/"


# Create Trajectory table
create a Geo data frame, with route/trajectory and entry_time into the grid_space relating to a sector. Route and time relates to the oldest flight plan entry of each flight. If trajectory.csv is already present, for the respective folder it will be loaded, created otherwise (this can take a while)*

In [3]:
grid_frame_dic =  {"sector_w_esmm": [13.5,56.5,19,60],"sector_67Y":[14,55,22,60]}
#TODO enter other sectors

In [4]:
a,b,c,d = grid_frame_dic[sector_name]
grid_frame =  box(a,b,c,d)

In [5]:
d_format = "%Y-%m-%dT%H:%M:%S.%f"
d_format_short = "%Y-%m-%dT%H:%M:%S"

class StartsInSectorException(Exception):
    pass

class NoExitFromSectorException(Exception):
    pass

class NoIntersectionException(Exception):
    pass

def get_entry(line, sector):
    intersection_line = line.intersection(sector)
    if isinstance(intersection_line, MultiLineString):
        intersection_line = intersection_line.geoms[0]
    intersections = intersection_line.xy
    if len(intersections[0]) == 0:
        raise NoIntersectionException
    entry_lon = intersections[0][0]
    entry_lat = intersections[1][0]
    return entry_lon, entry_lat

def get_entry_time(A,C,A_time_str,C_time_str,B):
    time_A = datetime.datetime.strptime(A_time_str,d_format_short) 
    time_C = datetime.datetime.strptime(C_time_str,d_format_short)
    time_B = time_A + datetime.timedelta(seconds=(time_C-time_A).total_seconds() * (A.distance(B) / A.distance(C)))
    return time_B.strftime(d_format)

def find_first_and_time(route,sector):
    points, times = [],[]
    for elem in route:
        points.append(Point(elem['lon'],elem['lat']))
        times.append(elem['eto'])
    if sector.contains(points[0]):
        return route[0]['lon'], route[0]['lat'],route[0]['eto']
    for j,p in enumerate(points[1:]):
        i = j-1
        line = LineString([points[i-1],p])
        if line.intersects(sector):
            entry_lon, entry_lat = get_entry(line, sector)
            entry_time = get_entry_time(points[i-1],p,times[i-1],times[i],Point(entry_lon, entry_lat))
            return entry_lon, entry_lat, entry_time
    raise NoIntersectionException

In [6]:
folders = []
for d in os.listdir(raw_data_path):
    if not d.endswith('.zip') and d != 'sectors_info':
        folders.append(raw_data_path+d+"/")

In [7]:
folders

['raw_data/scat20161015_20161021/',
 'raw_data/scat20161112_20161118/',
 'raw_data/scat20161210_20161216/',
 'raw_data/scat20170107_20170113/',
 'raw_data/scat20170215_20170221/',
 'raw_data/scat20170304_20170310/',
 'raw_data/scat20170401_20170407/',
 'raw_data/scat20170429_20170505/',
 'raw_data/scat20170527_20170602/',
 'raw_data/scat20170624_20170630/',
 'raw_data/scat20170722_20170728/',
 'raw_data/scat20170819_20170825/',
 'raw_data/scat20170916_20170922/']

In [8]:
if "trajectories.csv" not in os.listdir(f"{processed_data_path}/{sector_name}/intermediate_data/"):
    predicted_trajectories = []
    start =  time.time()
    for each_folder in folders:
        print(each_folder)
        for each_file in os.listdir(each_folder):
            if each_file in ['airspace.json',"grib_meteo.json"]:
                continue
            with open(each_folder + each_file, 'r') as f:
                flight = json.load(f)
            if len(flight['predicted_trajectory']) == 0:
                continue
            ind,pt = 0, flight['predicted_trajectory'][0]
            route_lst = []
            for p in pt['route']:
                route_lst.append(Point(p['lon'],p['lat']))
            try:
                entry_lon, entry_lat, entry_time = find_first_and_time(pt['route'],grid_frame)
            except NoIntersectionException:
                continue
            predicted_trajectories.append({'flightID':flight['id'], 'time_stamp':pt['time_stamp'], "entry_time":entry_time ,"index":ind, 
                                           "geometry":LineString(route_lst)})
        print(time.time() - start)
    crs = 'epsg:4326'
    flight_plan_gdf = gpd.GeoDataFrame(predicted_trajectories,crs = crs, geometry="geometry")
    flight_plan_gdf.to_csv(f"{processed_data_path}/{sector_name}/intermediate_data/trajectories.csv")


raw_data/scat20161015_20161021/
424.9291591644287
raw_data/scat20161112_20161118/
798.3067941665649
raw_data/scat20161210_20161216/
1635.1222078800201
raw_data/scat20170107_20170113/
1880.855153799057
raw_data/scat20170215_20170221/
2141.2517206668854
raw_data/scat20170304_20170310/
2424.3372168540955
raw_data/scat20170401_20170407/
2731.20641040802
raw_data/scat20170429_20170505/
3016.1796345710754
raw_data/scat20170527_20170602/
3655.6356797218323
raw_data/scat20170624_20170630/
4069.811628341675
raw_data/scat20170722_20170728/
4472.317538738251
raw_data/scat20170819_20170825/
4932.861466407776
raw_data/scat20170916_20170922/
5403.051256895065


In [9]:
pred_traj_pd = pd.read_csv(f"{processed_data_path}/{sector_name}/intermediate_data/trajectories.csv",index_col=0)
pred_traj_pd['geometry'] = pred_traj_pd['geometry'].apply(wkt.loads)

# Trajectory
creates matrix for every flight, which represents if a certain grid within the gridspace is ever crossed by the trajectory of a flight (refering to the trajectory.csv)

In [10]:
def create_grid(lon_min, lon_max, lonN, lat_min, lat_max, latN):
    lon_step_size = (lon_max - lon_min) / lonN
    lat_step_size = (lat_max - lat_min) / latN
    tiles = []
    lat = lat_max
    for i in range(latN):
        lon = lon_min
        lat_next = lat-lat_step_size
        for j in range(lonN):
            lon_next = lon+lon_step_size
            tiles.append(box(lon, lat, lon_next, lat_next))
            lon = lon_next 
        lat = lat_next
    grid_frame = box(lon_min,lat_min,lon_max,lat_max)
    return tiles , grid_frame 

In [11]:
a,b,c,d = grid_frame_dic[sector_name]
grid_cells,grid_frame = create_grid(a,c,10,b,d,10)
crs = 'epsg:4326'
grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs=crs)


In [12]:
%%time
filtered_path = f'./{processed_data_path}{sector_name}/sector_{sector_name}_buffer15_combined_results.csv'
df_filtered = pd.read_csv(filtered_path)
df_filtered['forecasted_entry_time'] = pd.to_datetime(df_filtered['forecasted_entry_time'])
df_filtered.head()

Wall time: 224 ms


,id,predicted_trajectory_ts,actual_entry_time,forecasted_entry_time,prediction_time,plt_entry_lon,plt_entry_lat,fp_entry_lon,fp_entry_lat,at_pred_lon,at_pred_lat,entry_deviation
0,100002,2016-10-20T19:33:13.914000,2016-10-20T19:49:03.898437,2016-10-20 19:48:37.086194,2016-10-20T19:33:37.086194,15.402479,57.226737,15.408764,57.223158,13.267107,55.698549,0.550407
1,100009,2016-10-16T12:53:23.931000,2016-10-16T13:08:54.093750,2016-10-16 13:09:22.733682,2016-10-16T12:54:22.733682,15.292028,57.289634,15.266457,57.304196,13.131639,55.932303,2.237535
2,100012,2016-10-19T19:10:22.579000,2016-10-19T19:26:29.296875,2016-10-19 19:26:05.576465,2016-10-19T19:11:05.576465,16.895720,57.406507,16.903628,57.410547,15.719317,55.563499,0.654471
3,100016,2016-10-20T20:44:09.172000,2016-10-20T20:59:23.898437,2016-10-20 20:59:23.719499,2016-10-20T20:44:23.719499,14.898149,57.513930,14.898420,57.513776,12.229291,56.407703,0.023667
4,100020,2016-10-18T10:04:23.831000,2016-10-18T10:19:48.695312,2016-10-18 10:19:50.148890,2016-10-18T10:04:50.148890,15.399915,57.228198,15.403606,57.226096,13.211440,55.661244,0.323228


In [13]:
relevant_ids = df_filtered['id'].values

pt_gdf = pred_traj_pd.loc[pred_traj_pd['flightID'].isin(relevant_ids)].copy()

pt_gdf['time_stamp'] = pd.to_datetime(pt_gdf['time_stamp'])

In [14]:
%%time
if "trajectory.csv" not in os.listdir(f"{processed_data_path}/{sector_name}/occupancy/"):
    print("start creating trajectory matrices")
    pt_occupancies = []
    for id_ , flight in df_filtered.iterrows():
        this_matrix = np.zeros(len(grid))
        if flight['id'] not in pt_gdf['flightID'].values:
            pt_occupancies.append(this_matrix)
            print(flight['id'],"has no grid intersection")
            continue
        route = pt_gdf.loc[pt_gdf['flightID'] == flight['id']]['geometry'].values[0]
        intersections = grid.intersects(route)
        this_matrix += intersections
        pt_occupancies.append(this_matrix)
    colnames = ["flight_id"] + list(range(len(grid)))
    pt_occ_df = pd.DataFrame(np.insert(pt_occupancies,0,df_filtered['id'].values,axis=1), columns =colnames)
    pt_occ_df.to_csv(f"{processed_data_path}/{sector_name}/occupancy/trajectory.csv",index=False)
    print("trajectory.csv created")
    pt_occ_df
else:
    print("trajectory.csv already existend for sector")

start creating trajectory matrices
1308320 has no grid intersection
trajectory.csv created
Wall time: 27.5 s


# Occupancy
Create the occupancy matrixes at different timepoints (see deltas_from_et). Variable refers to minutes after the expected sector entry of a flight.

To recreate the matrices ensure that there is an empty folder occupancy/ in the respective processed_data/sector directory. 

In [15]:
def point_at_time(trajectory, timestamp):
    last_point = None
    for point in trajectory:
        eto = datetime.datetime.strptime(point['eto'],"%Y-%m-%dT%H:%M:%S")
        if eto > timestamp and last_point:
            last_eto =  datetime.datetime.strptime(last_point['eto'],"%Y-%m-%dT%H:%M:%S")
            if eto == last_eto:
                continue
            factor = (timestamp -last_eto).seconds / (eto - last_eto).seconds
            res_lon = last_point['lon'] + factor*(point['lon'] - last_point['lon'])
            res_lat = last_point['lat'] + factor*(point['lat'] - last_point['lat'])
            return Point(res_lon, res_lat)
        else:
            last_point = point
    return None

def find_folder(path, file):
    for folder in os.listdir(path):
        if not folder.endswith('.zip'):
            if file in os.listdir(path+folder):
                return path+folder+"/"
    return ""  

In [16]:
pt_gdf['entry_time'] = pd.to_datetime(pt_gdf['entry_time'])
pt_gdf['time_stamp'] = pd.to_datetime(pt_gdf['time_stamp'])

In [17]:
relevant_pts_ff = {}
for id_ , flight in df_filtered.iterrows():
    et = flight['forecasted_entry_time']
    possible_pt = pt_gdf.loc[(pt_gdf['entry_time'] > et - datetime.timedelta(minutes = 30)) 
                             & (pt_gdf['entry_time'] < et + datetime.timedelta(minutes = 30))
                            # & (pt_gdf['time_stamp'] < et - timedelta(minutes = buffer))
                            & (pt_gdf['flightID'] != flight['id'])]
    most_recent_ones = possible_pt.groupby("flightID", as_index = False).max("time_stamp").reset_index()
    relevant_pts_ff[flight['id']] = most_recent_ones['flightID'].values

In [18]:
video_occ_path = f"{processed_data_path}/{sector_name}/occupancy/"
deltas_from_et = [0,5,10]

if "occs" not in os.listdir(video_occ_path):
    print("start creating occupancy matrices")
    os.mkdir(f"{video_occ_path}occs")
    c = 0
    point_delta_dict = {}
    for id_ , flight in df_filtered.iterrows():
        c+=1
        folder_path = find_folder(raw_data_path, str(flight['id']) + ".json")
        entry_time = flight['forecasted_entry_time']
        point_delta_dict[flight['id']] = {}
        for fid_pt in relevant_pts_ff[flight['id']]:
            point_delta_dict[flight['id']][fid_pt] = {}
            with open(folder_path+ str(fid_pt) +".json", 'r') as f:
                flight_json = json.load(f)
            pt_route = flight_json['predicted_trajectory'][0]['route']
            for delta in deltas_from_et:
                point = point_at_time(pt_route, entry_time + datetime.timedelta(minutes=delta))
                point_delta_dict[flight['id']][fid_pt][delta] = point
        if (c%1000)==0:
            print(c/len(df_filtered) * 100, "%")
            all_occs = []
            for fid, point_dict in point_delta_dict.items():
                occs = []
                for delta in deltas_from_et:
                    this_delta_occ = np.zeros(100)
                    for pt_ids,delta_dict in point_dict.items():
                        if delta_dict[delta]:
                            this_delta_occ += grid.intersects(delta_dict[delta])
                    occs.append(this_delta_occ)
                all_occs.append(occs)
            all_occs = np.array(all_occs)
            np.save(video_occ_path+"occs/"+str(c/1000),all_occs)
            point_delta_dict = {}
    all_occs = []
    for fid, point_dict in point_delta_dict.items():
        occs = []
        for delta in deltas_from_et:
            this_delta_occ = np.zeros(100)
            for pt_ids,delta_dict in point_dict.items():
                if delta_dict[delta]:
                    this_delta_occ += grid.intersects(delta_dict[delta])
            occs.append(this_delta_occ)
        all_occs.append(occs)
    all_occs = np.array(all_occs)
    np.save(video_occ_path+"occs/"+f"{int(len(df_filtered)//1000)+1}.0",all_occs)
    np.save(video_occ_path+"ids", df_filtered['id'].values)
    print("finished creating occupancy matrices")
else:
    print("occupancy matrices already exist for the sector. To recreate, delete occs and ids folder")

start creating occupancy matrices
4.878762745767673 %
9.757525491535347 %
14.636288237303019 %
19.515050983070694 %
24.393813728838367 %
29.272576474606037 %
34.151339220373714 %
39.03010196614139 %
43.90886471190906 %
48.787627457676734 %
53.6663902034444 %
58.545152949212074 %
63.42391569497975 %
68.30267844074743 %
73.1814411865151 %
78.06020393228277 %
82.93896667805043 %
87.81772942381812 %
92.6964921695858 %
97.57525491535347 %
finished creating occupancy matrices
